In [ ]:
# Dependencies
import requests
import time
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import pandas as pd
import json


In [ ]:
#Load Alpha Vantage API key from .env file
load_dotenv()
av_api_key = os.getenv("AV_API_KEY")
print(av_api_key)
 

In [ ]:
# create a datetime conversion function
def convert_to_datetime(time_stamp):
    date_format = "%Y%m%dT%H%M%S"
    return datetime.strptime(time_stamp, date_format)

In [ ]:
# Pull in the list of companies

df_SP_500 = pd.read_csv("constituents.csv")
ticker_list = df_SP_500['Symbol'].astype(str).tolist()
ticker_list = [t.replace('.', '-') for t in ticker_list]

for ticker in ticker_list:
    print(ticker)

In [21]:
# loop through the tickers
counter = 0

for ticker in ticker_list:
    # create a data frame
    df_tickers = pd.DataFrame()

    # Run a counter & time sleep to pause every 75 pulls
    counter = counter + 1
    counter_check = counter % 75
    # print(counter)

    if counter_check == 0:  
        time.sleep(60)

    # Building the Alpha Vantage API query URL to test if it works using IBM
    url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&symbol=' + ticker + '&apikey=' + "M9BTYXZI7EG8P00S"
    r = requests.get(url)
    data = r.json()

    # Convert JSON to DataFrame
    df_test = pd.DataFrame.from_dict(data['Time Series (Daily)'], orient='index')

    # Reset index to make the date keys a column
    df_test.reset_index(inplace=True)

    # Rename the index column to 'date'
    df_test.rename(columns={'index': 'date'}, inplace=True)

    # convert the date & time to a datetime field
    df_test['date'] = pd.to_datetime(df_test['date'])

    # drop the high, low, adjusted close, dividence and split coefficient columns
    df_test = df_test.drop(['1. open', '2. high', '3. low', '5. adjusted close', '7. dividend amount', '8. split coefficient'], axis=1)
    df_test.rename(columns={'4. close': 'close','6. volume': 'volume'}, inplace=True)

    # Add columns for the close price from one day and one week prior
    df_test['close_1_day_prior'] = df_test['close'].shift(-1)
    df_test['close_1_week_prior'] = df_test['close'].shift(-5)

    # Convert to an integer
    df_test['close'] = df_test['close'].astype(float)
    df_test['close_1_day_prior'] = df_test['close_1_day_prior'].astype(float)
    df_test['close_1_week_prior'] = df_test['close_1_week_prior'].astype(float)


    # Calculate the percent change compared to the prior day and prior week
    df_test['pct_change_1_day'] = df_test['close'].pct_change(periods=-1).astype(float)
    df_test['pct_change_1_week'] = df_test['close'].pct_change(periods=-5).astype(float)

    # filter for rows where the year is 2023
    df_test_2023 = df_test[df_test['date'].dt.year == 2023]
    # Reset index and drop the old index
    df_test_2023.reset_index(drop=True, inplace=True)

    # Create an export location and export the data frame to a CSV
    file_path = '2023_Closing_Stock_Prices/' + ticker + '_stock_data.csv'
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    df_test_2023.to_csv(file_path, index=False)

    print(f"DataFrame exported to {file_path}")


    

DataFrame exported to 2023_Closing_Stock_Prices/AVGO_stock_data.csv
DataFrame exported to 2023_Closing_Stock_Prices/BR_stock_data.csv
DataFrame exported to 2023_Closing_Stock_Prices/BRO_stock_data.csv
DataFrame exported to 2023_Closing_Stock_Prices/BF-B_stock_data.csv
DataFrame exported to 2023_Closing_Stock_Prices/BLDR_stock_data.csv
DataFrame exported to 2023_Closing_Stock_Prices/BG_stock_data.csv
DataFrame exported to 2023_Closing_Stock_Prices/BXP_stock_data.csv
DataFrame exported to 2023_Closing_Stock_Prices/CHRW_stock_data.csv
DataFrame exported to 2023_Closing_Stock_Prices/CDNS_stock_data.csv
DataFrame exported to 2023_Closing_Stock_Prices/CZR_stock_data.csv
DataFrame exported to 2023_Closing_Stock_Prices/CPT_stock_data.csv
DataFrame exported to 2023_Closing_Stock_Prices/CPB_stock_data.csv
DataFrame exported to 2023_Closing_Stock_Prices/COF_stock_data.csv
DataFrame exported to 2023_Closing_Stock_Prices/CAH_stock_data.csv
DataFrame exported to 2023_Closing_Stock_Prices/KMX_stock_d